In [ ]:
# TODO: requirements.txt
# pip install binreader
# pip install matplotlib
# pip install numpy
# pip install nbconvert

In [ ]:
from typing import Callable

# Import
from pupil_info import *
import matplotlib.pyplot as plt
import numpy as np
from local_features import *
from recording import Measure, MidiNote, Recording
from typing import Callable
from matplotlib.axes import Axes
from matplotlib.figure import Figure


INFO: list[PupilInfo] = get_all_pupil_info()

In [ ]:
# Piano roll for a specified performance
from recording import OutputName

performance: PerformanceInfo = INFO[2].sessions[0].performances[2]
print(SONG_NAMES[performance.song], "with", ALGORITHM_NAMES[performance.algorithm])
recording = performance.recording
midi_notes: list[MidiNote] = all_notes(recording.measures)


fig: Figure
ax: Axes
fig, ax = plt.subplots(figsize=(20, 5))

# Plot each note with the corresponding color
for note in midi_notes:
    color = "red" if note.output_name == OutputName.LOOPBACK else "blue"
    ax.broken_barh([(note.time, note.length)], (note.note - 0.4, 0.8), facecolors=color)

# Formatting
ax.invert_yaxis()
ax.set_xticks(np.arange(0, 64, 4))
ax.set_yticks([])

ax.grid(True)

# Add legend
legend_patches = [
    plt.Line2D([0], [0], color="red", lw=4, label="Loopback"),
    plt.Line2D([0], [0], color="blue", lw=4, label="Algorithm")
]
ax.legend(handles=legend_patches, loc="upper right")

plt.show()

In [ ]:
# Are some songs simply more fun than others?

# Evaluate this specified question
question_index: int = 3

# Get all performances
all_performances: list[PerformanceInfo] = [
    INFO[pupil_index].sessions[session_index].performances[performance_index]
    for pupil_index in range(NUM_PUPILS)
    for session_index in range(NUM_SESSIONS)
    for performance_index in range(NUM_PERFORMANCES)
]

# Keep track of all scores
all_scores: list[list[int]] = []

song_index: int
for song_index in range(NUM_SONGS):
    # Get all scores
    scores: list[int] = [performance.responses[question_index] for performance in all_performances if performance.song == song_index]

    # Append to plotting variable
    all_scores.append(scores)

fig: Figure
ax: Axes
fig, ax = plt.subplots()
ax.violinplot(all_scores, showmeans=True)
ax.set_title('Enjoyability of songs')

ax.yaxis.grid(True)
ax.set_ylim([0.8, 5.2])
_ = ax.set_xticks([y + 1 for y in range(NUM_SONGS)], labels=list(SONG_IDS.keys()))
ax.set_yticks(range(1, 6))

print([np.average(s) for s in all_scores])

plt.savefig('charts/song_enjoyability.png')

In [ ]:
# Are some algorithms simply more fun than others?

# Evaluate this specified question and algorithm
question_index: int = 3

# Get all performances
all_performances: list[PerformanceInfo] = [
    INFO[pupil_index].sessions[session_index].performances[performance_index]
    for pupil_index in range(NUM_PUPILS)
    for session_index in range(NUM_SESSIONS)
    for performance_index in range(NUM_PERFORMANCES)
]

all_scores: list[list[int]] = []

algorithm_index: int
for algorithm_index in range(NUM_ALGORITHMS):
    # Get scores of performances of algorithm
    scores: list[int] = [
        performance.responses[question_index]
        for performance in all_performances
        if performance.algorithm == algorithm_index
    ]

    all_scores.append(scores)

fig: Figure
ax: Axes
fig, ax = plt.subplots()
ax.violinplot(all_scores, showmeans=True)
ax.set_title('Enjoyability of algorithms')

ax.yaxis.grid(True)
ax.set_ylim([0.8, 5.2])
_ = ax.set_xticks([y + 1 for y in range(NUM_ALGORITHMS)], labels=list(ALGORITHM_IDS.keys()))
ax.set_yticks(range(1, 6))

print([np.average(s) for s in all_scores])

plt.savefig('charts/algorithm_enjoyability.png')

In [ ]:
# Get all performances
all_performances: list[PerformanceInfo] = [
    INFO[pupil_index].sessions[session_index].performances[performance_index]
    for pupil_index in range(NUM_PUPILS)
    for session_index in range(NUM_SESSIONS)
    for performance_index in range(NUM_PERFORMANCES)
]

fig, axs = plt.subplots(2)

colors = ['red', 'green', 'blue']

fig.subplots_adjust(hspace=0.5)

width = 0.25

for algorithm_index in range(NUM_ALGORITHMS):
    # Get scores of performances of algorithm
    performances: list[PerformanceInfo] = [
        performance
        for performance in all_performances
        if performance.algorithm == algorithm_index
    ]

    algorithm_respond_scores: list[int] = [sum([p.responses[0] == v for p in performances]) for v in range(1, 6)]
    axs[0].title.set_text("\"I feel that the algorithm could respond well to me. \"")
    axs[0].bar(x=np.array(range(1, 6)) + width * algorithm_index,height=algorithm_respond_scores, width=width, color=colors[algorithm_index])
    axs[0].set_yticks(range(0, 15, 5))

    pupil_respond_scores: list[int] = [sum([p.responses[1] == v for p in performances]) for v in range(1, 6)]
    axs[1].title.set_text("\"I feel that I could respond well to the algorithm.\"")
    axs[1].bar(x=np.array(range(1, 6)) + width * algorithm_index,height=pupil_respond_scores, width=width, color=colors[algorithm_index])
    axs[1].set_yticks(range(0, 15, 5))

plt.savefig('charts/response_scores.png')

In [ ]:
# Are pupils self-reporting feeling more musical over time?
# Are pupils self-reporting feeling more response (from themselves; from the algorithm) over time?
# Are pupils self-reporting feeling more enjoyment over time?

# Evaluate this specified question
# 0: I feel that the algorithm could respond well to me.
# 1: I feel that I could respond well to the algorithm.
# 2: I feel that the algorithm gives me inspiration.
# 3: I enjoyed playing this song with the algorithm.
question_index: int = 3

session_index: int
for session_index in range(NUM_SESSIONS):
    # Get all performances of the session
    performances: list[PerformanceInfo] = [
        INFO[pupil_index].sessions[session_index].performances[performance_index]
        for pupil_index in range(NUM_PUPILS)
        for performance_index in range(NUM_PERFORMANCES)
    ]

    # Get the question scores
    scores: list[int] = [
        performance.responses[question_index]
        for performance in performances
    ]

    # (TODO: remove) Filter all not-yet-filled-in answers
    scores = [score for score in scores if score > 0]

    print(session_index, ": ", np.average(scores), sep='')


In [ ]:
# For a specific algorithm, for a specific metric, how does it (and its distribution) evolve over the sessions?

# Evaluate this specific metric
metric: Callable[[list[Measure]], float]
metric = melodic_arc_duration_avg

# Evaluate this specific algorithm
algorithm_index: int = 0

all_metrics: list[list[float]] = []

session_index: int
for session_index in range(NUM_SESSIONS):
    # Get performances
    recordings: list[Recording] = [
        INFO[pupil_index].sessions[session_index].performances[performance_index].recording
        for pupil_index in range(NUM_PUPILS)
        for performance_index in range(NUM_PERFORMANCES)
    ]

    # Get metrics
    metrics: list[float] = [
        metric(four)
        for recording in recordings
        for four in recording.fours
    ]

    all_metrics.append(metrics)

fig: Figure
ax: Axes
fig, ax = plt.subplots()
ax.violinplot(all_metrics, showmeans=True)
ax.set_title('Violin plot')

ax.yaxis.grid(True)

ax.set_xticks([1, 2, 3])

plt.show()

In [ ]:
# (out of personal interest) Is the fifth really more popular in solos of Summertime?

from functools import partial

all_metrics: list[list[float]] = []

for song_index in range(NUM_SONGS):
    fifth = SONG_PITCH_CLASSES[SONG_NAMES[song_index]][2]

    # Get performances
    performances = [
        INFO[pupil_index].sessions[session_index].performances[performance_index]
        for pupil_index in range(NUM_PUPILS)
        for session_index in range(NUM_SESSIONS)
        for performance_index in range(NUM_PERFORMANCES)
    ]

    # Filter by song
    performances = [
        performance
        for performance in performances
        if performance.song == song_index
    ]

    # Get all recordings
    recordings: list[Recording] = [performance.recording for performance in performances]

    # Get metrics
    metrics: list[float] = [
        note_share(four, pitch_class=fifth)
        for recording in recordings
        for four in recording.fours
    ]

    all_metrics.append(metrics)


fig: Figure
ax: Axes
fig, ax = plt.subplots()
ax.violinplot(all_metrics, showmeans=True)
ax.set_title('Violin plot')

ax.yaxis.grid(True)
_ = ax.set_xticks([y + 1 for y in range(NUM_SONGS)], labels=list(SONG_IDS.keys()))

plt.show()

In [ ]:
# For a specific question, how does its self-reporting score relate to the expert evaluation?

# TODO (of course)